In [4]:
pip install scikit-learn

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 KB 6.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 30.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.2/103.2 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 37.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.2 MB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
os.chdir("/Users/AAYAN-LAPTOP/Downloads/dataset and all")

In [12]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt

# Load the datasets
train_features = pd.read_csv('training_set_features.csv')
train_labels = pd.read_csv('training_set_labels.csv')
test_features = pd.read_csv('test_set_features.csv')
submission_format = pd.read_csv('submission_format.csv')

# Fill missing values
train_features.fillna(train_features.mode().iloc[0], inplace=True)
test_features.fillna(test_features.mode().iloc[0], inplace=True)

In [14]:
# Identify categorical features
categorical_features = ['age_group', 'education', 'race', 'sex', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation']

# Initialize OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the training data
encoded_train_features = encoder.fit_transform(train_features[categorical_features])

# Transform the test data
encoded_test_features = encoder.transform(test_features[categorical_features])

# Convert encoded features back to DataFrame
encoded_train_features = pd.DataFrame(encoded_train_features, index=train_features.index)
encoded_test_features = pd.DataFrame(encoded_test_features, index=test_features.index)

# Drop original categorical columns and concatenate the encoded columns
train_features = train_features.drop(columns=categorical_features).reset_index(drop=True)
test_features = test_features.drop(columns=categorical_features).reset_index(drop=True)

# Concatenate the encoded features with the rest of the features
train_features = pd.concat([train_features, encoded_train_features], axis=1)
test_features = pd.concat([test_features, encoded_test_features], axis=1)

In [16]:
# Ensure that the train and test features have the same columns
train_features, test_features = train_features.align(test_features, join='left', axis=1, fill_value=0)

# Convert column names to strings
train_features.columns = train_features.columns.astype(str)
test_features.columns = test_features.columns.astype(str)

In [17]:
# Train the models
model_xyz = LogisticRegression(max_iter=1000)
model_seasonal = LogisticRegression(max_iter=1000)

model_xyz.fit(train_features, train_labels['xyz_vaccine'])
model_seasonal.fit(train_features, train_labels['seasonal_vaccine'])

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

LogisticRegression(max_iter=1000)

In [19]:
# Predict probabilities on the training set
train_pred_xyz = model_xyz.predict_proba(train_features)[:, 1]
train_pred_seasonal = model_seasonal.predict_proba(train_features)[:, 1]

# Calculate ROC AUC scores
roc_auc_xyz = roc_auc_score(train_labels['xyz_vaccine'], train_pred_xyz)
roc_auc_seasonal = roc_auc_score(train_labels['seasonal_vaccine'], train_pred_seasonal)

print(f'ROC AUC for xyz_vaccine: {roc_auc_xyz}')
print(f'ROC AUC for seasonal_vaccine: {roc_auc_seasonal}')

ROC AUC for xyz_vaccine: 0.8366476276491239
ROC AUC for seasonal_vaccine: 0.8545507321930541


In [20]:
# Predict probabilities on the test set
test_pred_xyz = model_xyz.predict_proba(test_features)[:, 1]
test_pred_seasonal = model_seasonal.predict_proba(test_features)[:, 1]

In [21]:
# Prepare the submission file
submission_format['xyz_vaccine'] = test_pred_xyz
submission_format['seasonal_vaccine'] = test_pred_seasonal

# Save the submission file
submission_format.to_csv('submission.csv', index=False)